# Process Nino 3.4 dataset from monthly to annual means, with different year definitions

## Load data

In [3]:
import pandas as pd

In [19]:
n34 = pd.read_csv("nina34.anom.data.txt", delim_whitespace=True)
n34.head()

,year,01,02,03,04,05,06,07,08,09,10,11,12
0,1950,-1.99,-1.69,-1.42,-1.54,-1.75,-1.27,-1.01,-0.97,-0.98,-1.03,-1.23,-1.31
1,1951,-1.30,-1.04,-0.38,-0.23,-0.01,0.00,0.30,0.17,0.51,0.49,0.55,0.31
2,1952,0.13,-0.01,-0.11,-0.02,-0.14,-0.54,-0.76,-0.56,-0.36,-0.46,-0.78,-0.39
3,1953,0.20,0.24,0.29,0.22,0.35,0.39,0.14,0.09,0.29,0.16,0.18,0.41
4,1954,0.43,0.28,-0.38,-1.17,-0.81,-0.93,-1.18,-1.43,-1.60,-1.48,-1.13,-1.33


In [23]:
n34.year.head()

0    1950
1    1951
2    1952
3    1953
4    1954
Name: year, dtype: int64

Munge

In [32]:
n34l = n34.melt(id_vars=['year'], value_vars=['01','02','03','04','05','06','07','08','09','10','11','12'], var_name='month', value_name='nino34')

In [ ]:
pd.to_numeric(n34l.month)

In [58]:
n34l = n34l[n34l.nino34 != -99.99]

## Timey Wimey stuff

In [37]:
n34l['date'] = n34l.year.astype(str)+n34l.month.astype(str)

In [38]:
n34l

,year,month,nino34,date
0,1950,01,-1.99,195001
1,1951,01,-1.30,195101
2,1952,01,0.13,195201
3,1953,01,0.20,195301
4,1954,01,0.43,195401
...,...,...,...,...
859,2017,12,-0.98,201712
860,2018,12,0.89,201812
861,2019,12,0.52,201912
862,2020,12,-1.15,202012


In [48]:
n34l['datetime'] = pd.to_datetime(n34l.date, format='%Y%m')

In [49]:
n34l

,year,month,nino34,date,datetime
0,1950,01,-1.99,195001,1950-01-01
1,1951,01,-1.30,195101,1951-01-01
2,1952,01,0.13,195201,1952-01-01
3,1953,01,0.20,195301,1953-01-01
4,1954,01,0.43,195401,1954-01-01
...,...,...,...,...,...
859,2017,12,-0.98,201712,2017-12-01
860,2018,12,0.89,201812,2018-12-01
861,2019,12,0.52,201912,2019-12-01
862,2020,12,-1.15,202012,2020-12-01


In [52]:
n34l['north'] = n34l['datetime'].dt.to_period('A-NOV')

In [54]:
n34l['south'] = n34l['datetime'].dt.to_period('A-MAY')

In [55]:
n34l['tropic'] = n34l['datetime'].dt.to_period('A-APR')

In [56]:
n34l

,year,month,nino34,date,datetime,north,south,tropic
0,1950,01,-1.99,195001,1950-01-01,1950,1950,1950
1,1951,01,-1.30,195101,1951-01-01,1951,1951,1951
2,1952,01,0.13,195201,1952-01-01,1952,1952,1952
3,1953,01,0.20,195301,1953-01-01,1953,1953,1953
4,1954,01,0.43,195401,1954-01-01,1954,1954,1954
...,...,...,...,...,...,...,...,...
859,2017,12,-0.98,201712,2017-12-01,2018,2018,2018
860,2018,12,0.89,201812,2018-12-01,2019,2019,2019
861,2019,12,0.52,201912,2019-12-01,2020,2020,2020
862,2020,12,-1.15,202012,2020-12-01,2021,2021,2021


## Aggregate

Northern Hemisphere (Dec-Dec)

In [93]:
var = 'north'
(n34l[[var,'nino34']]
 .rename(columns={var: 'year'})
 .groupby('year')
 .mean()
 .to_csv(f'nino34_annual_{var}.csv'))

Southern Hemisphere (Jun-Jun)

In [94]:
var = 'south'
(n34l[[var,'nino34']]
 .rename(columns={var: 'year'})
 .groupby('year')
 .mean()
 .to_csv(f'nino34_annual_{var}.csv'))

Tropical year (May-May)

In [95]:
var = 'tropic'
(n34l[[var,'nino34']]
 .rename(columns={var: 'year'})
 .groupby('year')
 .mean()
 .to_csv(f'nino34_annual_{var}.csv'))

Hsiang 2011 - "Civil conflicts are associated with the global climate" (Mar-Dec) <br>
https://www.nature.com/articles/nature10311

In [106]:
(n34l[n34l.month > 4]
 .groupby('year')
 .mean()
 .nino34
 .to_csv(f'nino34_annual_hsiang2011.csv'))